In [1]:
# the first scrapes I want are of 5 stock tickers from yahoo finance API
# stocks: Airbnb, Tesla, Crowdstrike, Palo Alto, Salesforce

In [29]:

#import the required libraries:
from requests import get
from bs4 import BeautifulSoup
import os
import requests
import json
from typing import Dict, List, Optional, Union, cast
import time
import pandas as pd
from datetime import date

#from env import yahoo_token, yahoo_username

In [30]:
# Create the pbjects to use later on:

url = 'https://finance.yahoo.com/'
endpoints = ['quote/ABNB?p=ABNB', 'quote/TSLA?p=TSLA', 'quote/CRWD?p=CRWD', 'quote/PANW?p=PANW', 'quote/CRM?p=CRM']
# headers will custom is key is required for API
headers = {'User-Agent': 'Codeup Data Science'}


In [31]:
# Write the response variable
def get_response():
    responses = []
    for endpoint in endpoints:
        response = get(url+endpoint, headers=headers)
        responses.append(response)
        response = response.text
        print(response[:15])
    return responses

In [32]:
get_response()

<!DOCTYPE html>
<!DOCTYPE html>
<!DOCTYPE html>
<!DOCTYPE html>
<!DOCTYPE html>


[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [ ]:
# bits and pieces test:
# set out the basic elements for the scraper to access
baseUrl = 'https://finance.yahoo.com/'
# add in the endpoints for the various stocks you want to check
endpoint = 'quote/ABNB?p=ABNB'
actual = 'https://finance.yahoo.com/quote/ABNB?p=ABNB'
# headers will be generic unless a custom is key is required for the yfinance API
#headers = {'User-Agent': 'Codeup Data Science'}
url = f'{baseUrl}{endpoint}'
#url = actual
response = get(url)
# create the scraper
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
response.text[:50]

'<!DOCTYPE html><html data-color-theme="light" id="'

In [ ]:
# BS4 custom tags
# used an atttribute to tell BS that we want to use for ID
# beatiful soup custom tags for more info
# Selenium for site interaction, but can be a little slow
# read in the old CSV, then add in the new, then overwrite
openingPrice = soup.find(attrs = {"data-test":"OPEN-value"}).text
ticker = soup.find(class_="D(ib) Fz(18px)").text
dailyRange = soup.find(attrs = {"data-test":"DAYS_RANGE-value"}).text
closingPrice = soup.find(attrs = {"data-test":"qsp-price", "data-field":"regularMarketPrice"}).text
openingPrice, ticker, dailyRange, closingPrice

('115.89', 'Airbnb, Inc. (ABNB)', '115.42 - 117.95', '117.03')

In [52]:
# I want date, open, high, close, stock name
def get_prices():
    # set out the basic elements for the scraper to access
    baseUrl = 'https://finance.yahoo.com/'
    # add in the endpoints for the various stocks you want to check
    endpoints = ['quote/ABNB?p=ABNB', 'quote/TSLA?p=TSLA', 'quote/CRWD?p=CRWD', 'quote/PANW?p=PANW', 'quote/CRM?p=CRM']
    # headers will be generic unless a custom is key is required for the yfinance API
    headers = {'User-Agent': 'Codeup Data Science'}

    #create an empty list to store the dictionary of daily values
    dailyOverview = []

    # make a for loop to iterate through the URL and endpoints
    for endpoint in endpoints:
        url = f'{baseUrl}{endpoint}'
        headers = {'User-Agent': 'Codeup Data Science'}
        while True:
            response = get(url, headers=headers)
            if response.ok:
                break
            else:
                time.sleep(15)

        # create the scraper
        soup = BeautifulSoup(response.text, 'html.parser')

        # list the variables to scrape

        day = date.today()

        ticker = soup.find(class_="D(ib) Fz(18px)").text

        openingPrice = soup.find(attrs = {"data-test":"OPEN-value"}).text

        dailyRange = soup.find(attrs = {"data-test":"DAYS_RANGE-value"}).text

        closingPrice = soup.find(attrs = {"data-test":"qsp-price", "data-field":"regularMarketPrice"}).text

        # create a dictionary to store the variables
        all_prices = {'Day': day,
            'Stock': ticker,
            'Open': openingPrice,
            'Range': dailyRange,
           'Close': closingPrice}

        # add a check statement to display the progress
        print(f'\rFetching page {endpoint}', end='')

        # append the iterated stock dictionary to the master list
        dailyOverview.append(all_prices)

    # convert the list to a DataFrame
    dailyOverview = pd.DataFrame(dailyOverview)

    # return the DataFrame
    return dailyOverview

In [53]:
df = get_prices()
df

Fetching page quote/CRM?p=CRMNW

,Day,Stock,Open,Range,Close
0,2022-08-25,"Airbnb, Inc. (ABNB)",115.89,115.42 - 117.95,117.03
1,2022-08-25,"Tesla, Inc. (TSLA)",302.36,291.63 - 302.95,296.07
2,2022-08-25,"CrowdStrike Holdings, Inc. (CRWD)",199.29,197.54 - 203.67,202.94
3,2022-08-25,"Palo Alto Networks, Inc. (PANW)",570.00,568.38 - 578.79,575.58
4,2022-08-25,"Salesforce, Inc. (CRM)",168.58,165.56 - 174.14,173.91
